In [ ]:
!pip install git+https://github.com/qubvel/classification_models.git
from classification_models.tfkeras import Classifiers
from PIL import Image
import os
import tensorflow as tf
import pandas as pd
import keras
from keras import layers, regularizers
import numpy as np
from sklearn.model_selection import train_test_split
from keras.applications.densenet import DenseNet169, preprocess_input
from keras.callbacks import EarlyStopping, ModelCheckpoint, LearningRateScheduler, TensorBoard

In [ ]:
import os
import cv2
import numpy as np
import pandas as pd
from datetime import datetime
counter = 0

def roi_clahe_pre_process(path, output_directory):
    global counter  # Access the global counter variable

    img = cv2.imread(path)

    # Convert image to grayscale
    gray = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)

    # Apply thresholding
    thresh = cv2.threshold(gray, 0, 255, cv2.THRESH_BINARY + cv2.THRESH_OTSU)[1]

    # Determine bounding rectangle of the thresholded image
    x, y, w, h = cv2.boundingRect(thresh)
    x, y, w, h = x, y, w + 20, h + 20

    # Crop the original image based on the bounding rectangle
    img = img[y:y+h, x:x+w]

    # Convert the cropped image to grayscale
    gray_crop = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)

    # Apply CLAHE
    clahe = cv2.createCLAHE(clipLimit=2.0, tileGridSize=(8, 8))
    clahe_img = clahe.apply(gray_crop)

    # Construct the output file path
    filename = os.path.basename(path)
    output_filename = filename[:-4] + '__enhanced_' + str(counter) + filename[-4:]
    output_path = os.path.join(output_directory, output_filename)

    # Save the output image
    cv2.imwrite(output_path, clahe_img)
    print(f"Image saved to {output_path}")

    # Increment the counter
    counter += 1

    return output_path


prefix = '/kaggle/input/better-mura/MURA-v1.1/'
output_directory = '/kaggle/working'
excel_file_path = '/kaggle/input/better-mura/MURA-v1.1/MURA-v1.1/train_augmented.csv'
df = pd.read_csv(excel_file_path)
df = df[df["path"].str.contains(r"XR_SHOULDER")]
paths = prefix + df['path'].astype(str)
labels = df['label']
labels = pd.Series(labels, name='Label')
labels = labels.astype(str)
t_images = pd.concat([paths, labels], axis=1)
print(t_images)
for index, row in t_images.iterrows():
    image_path = row["path"]
    label = row["Label"]
    enhanced = roi_clahe_pre_process(image_path , output_directory)
    #output_path=stat_range(image_path)
    # Create a DataFrame with the original and equalized image paths
    df_output = pd.DataFrame({'Original Path': [image_path], 'Enhanced Path': [enhanced], 'Label': [label]})
        # Append the DataFrame to the output CSV file
    if not os.path.isfile('/kaggle/working/Enhanced_Labeled.csv'):
            df_output.to_csv('/kaggle/working/Enhanced_Labeled.csv', index=False)
    else:
            df_output.to_csv('/kaggle/working/Enhanced_Labeled.csv', mode='a', header=False, index=False)
print("===============Done with training images===================")
valid_data = pd.read_csv('/kaggle/input/better-mura/MURA-v1.1/MURA-v1.1/valid.csv')
valid_data = valid_data[valid_data["BodyPart"].str.contains(r"XR_SHOULDER")]
paths = prefix + valid_data['path'].astype(str)
labels = valid_data['label']
labels = pd.Series(labels, name='Label')
labels = labels.astype(str)
v_images = pd.concat([paths, labels], axis=1)
print(v_images)
for index, row in v_images.iterrows():
    image_path = row["path"]
    label = row["Label"]
    enhanced = roi_clahe_pre_process(image_path , output_directory)
    #output_path=stat_range(image_path)
    # Create a DataFrame with the original and equalized image paths
    df_output = pd.DataFrame({'Original Path': [image_path], 'Enhanced Path': [enhanced], 'Label': [label]})
        # Append the DataFrame to the output CSV file
    if not os.path.isfile('/kaggle/working/Valid_Enhanced_Labeled.csv'):
            df_output.to_csv('/kaggle/working/Valid_Enhanced_Labeled.csv', index=False)
    else:
            df_output.to_csv('/kaggle/working/Valid_Enhanced_Labeled.csv', mode='a', header=False, index=False)
print("===============Done with test images===================")

In [ ]:
class_num=1
batch_size = 32

os.environ['TF_CPP_MIN_LOG_LEVEL'] = '2'

# 1. Load the images and labels

#training Data Loading
prefix = '/kaggle/input/better-mura/MURA-v1.1/'
train_data = pd.read_csv('/kaggle/working/Enhanced_Labeled.csv')


paths = train_data['Enhanced Path'].astype(str)
labels = train_data['Label']
labels = pd.Series(labels, name='Label')
labels = labels.astype(str)
t_images = pd.concat([paths, labels], axis=1)
print(t_images)
train_df, test_df = train_test_split(t_images, train_size=0.8, shuffle=True, random_state=1)
#testing Data Loading
valid_data = pd.read_csv('/kaggle/working/Valid_Enhanced_Labeled.csv')
paths = valid_data['Enhanced Path'].astype(str)
labels = valid_data['Label']
labels = pd.Series(labels, name='Label')
labels = labels.astype(str)
v_images = pd.concat([paths, labels], axis=1)
print(v_images)
# 2. Augmentation

from tensorflow.keras.preprocessing.image import ImageDataGenerator

# Define augmentation parameters
datagen = ImageDataGenerator(
    rescale=1./255,
    rotation_range=40,
    width_shift_range=0.2,
    height_shift_range=0.2,
    shear_range=0.2,
    zoom_range=0.2,
    horizontal_flip=True,
    fill_mode='nearest'
)

In [ ]:
#creating batch loaders to size :
imgsize = 224
xcol = "Enhanced Path"
Train_image_generator = datagen.flow_from_dataframe(
    dataframe=train_df,
    directory=None,  # Directory is None because we're providing full paths in image_paths
    x_col = xcol,
    y_col="Label",
    target_size=(imgsize,imgsize),  # Specify your image dimensions
    batch_size=batch_size,
    class_mode="binary"  # Assuming binary classification, change if needed
)

Test_image_generator = datagen.flow_from_dataframe(
    dataframe=test_df,
    directory=None,  # Directory is None because we're providing full paths in image_paths
    x_col = xcol,
    y_col="Label",
    target_size=(imgsize,imgsize),  # Specify your image dimensions
    batch_size=batch_size,
    class_mode="binary"  # Assuming binary classification, change if needed
)
valid_generator = datagen.flow_from_dataframe(
    dataframe=v_images, directory=None,
    x_col = xcol,
    y_col="Label",
    target_size=(imgsize,imgsize),
    class_mode="binary",
    batch_size=batch_size,
    shuffle=False)

In [ ]:
from keras.models import Model
from keras.layers import Input, Flatten, BatchNormalization, Dense, Activation, Dropout
from keras.applications import Xception
from keras.models import load_model
import shutil

# Source path of the model
source_path = "/kaggle/input/xception-shoulder/keras/xception-shoulder/3/MobilenetV2_shoulder.keras"
# Destination path where you can write
destination_path = "/kaggle/working/model.keras"

# Copy the model file
shutil.copyfile(source_path, destination_path)

# Now load the model from the copied file
model = load_model(destination_path)


In [ ]:
X_val = []
y_val = []

for index, row in v_images.iterrows():
    image_path = row["Enhanced Path"]
    label = row["Label"]

    # Load and preprocess the image
    image = Image.open(image_path).resize((imgsize, imgsize)).convert('RGB')
    image = np.array(image) / 255.0

    # Append the preprocessed image and label to X_val and y_val lists
    X_val.append(image)
    y_val.append(label)

# Convert lists to numpy arrays
X_val = np.array(X_val)
y_val = np.array(y_val)
y_val = y_val.astype(int)
# Evaluate the model on the entire validation dataset
loss, accuracy = model.evaluate(X_val, y_val)
print(f"Loss: {loss}")
print(f"Accuracy: {accuracy}")


from keras.models import load_model
model_1 = load_model('/kaggle/working/enhanced_densenet_169_shoulder.keras')